In [118]:
import pandas as pd
import numpy as np

In [252]:
full_df = pd.read_csv('/Users/AngelaTam/Downloads/inputed50_cleaned.csv')
ref_df = pd.read_csv('/Users/AngelaTam/Downloads/inputed50_ref.csv')
d1d2 = pd.read_csv('/Users/AngelaTam/Downloads/tadpole_challenge/TADPOLE_D1_D2.csv')

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720

In [ ]:
full_df.drop(full_df.columns[0],axis=1,inplace=True)

In [253]:
missing_ids = [x for x in ref_df.RID.unique() if any([not pd.notnull(y) for y in ref_df[ref_df.RID==x]['y_DX']])]
for i,sub in enumerate(missing_ids):
    jnk = ref_df[ref_df.RID==sub]['y_DX'].fillna(method='ffill')
    ref_df.loc[ref_df.RID==sub,'y_DX'] = jnk.values

In [254]:
full_df = full_df.loc[ref_df.dropna().index].reset_index(drop=True)
ref_df = ref_df.loc[ref_df.dropna().index].reset_index(drop=True)

In [ ]:
code = {1: 0, 2:1, 3:2}
ref_df['y_DX'] = [code[x] for x in ref_df['y_DX']]

In [256]:
full_df['RID'] = ref_df['RID']
full_df['VISCODE'] = ref_df['VISCODE']
full_df['COLPROT'] = ref_df['COLPROT']
full_df['EXAMDATE'] = ref_df['EXAMDATE']
full_df['Month_bl'] = d1d2['Month_bl']

In [257]:
data = full_df

target = ref_df[['y_DX','y_ADAS13','y_Ventricles_adj_TIV']]
#target = target.dropna(axis=0)
#data = data.loc[target.index]
#data = data.dropna(axis=1)
#target = target.loc[data.index]
target['Month_bl'] = data['Month_bl']
target = target[['Month_bl','y_DX','y_ADAS13','y_Ventricles_adj_TIV']]

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [258]:
for col in target.columns[1:]:
    data[col] = target[col]

In [260]:
#selection = data.iloc[sel_index][['RID','Month_bl']]

In [261]:

def compute_data_table(sel_index):
    x_ = []
    y_ = []
    selection = data.iloc[sel_index][['RID','Month_bl']]
    for i in range(selection.shape[0]):
        rid, vis_month = selection.iloc[i].values
        #print(rid,vis_month)
        mask = (data[data['RID']==rid]['Month_bl']>vis_month).values

        valid_data = data[data['RID']==rid][mask]
        target_ = target[data['RID']==rid][mask]
        
        #print(mask)
        if valid_data.shape[0]>0:
            # add to x 
            x_.append(data.iloc[sel_index[i]].drop(['VISCODE','EXAMDATE','COLPROT','Month_bl']).values)
            # add to y
            target_ = target_.values
            target_[:,0] = target_[:,0]-vis_month
            #print(target_)
            
            y_.append(target_)
            
    return x_,y_
 


In [262]:
sel_index = data.index
x_,y_ = compute_data_table(sel_index) 